## Download and Clean Temp/Precip Data

### Setup

In [11]:
import numpy as np
import pandas as pd
import requests
import os
import time
from bs4 import BeautifulSoup
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

In [12]:
df = pd.read_csv('individual-datasets/all-county-data-no-tmpprecip.csv')
counties = [str(county)[2:] for county in df.County.unique()]

In [15]:
for parameter, file_tag in {'tavg': '_t', 'pcp': '_p'}.items():
    for i in range(len(counties)):
        county = 'VA-'+counties[i]
        url = f'https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/county/time-series/{county}/{parameter}/1/0/2008-2022.csv'
        driver = webdriver.Chrome()
        driver.get(url)
        # download = driver.find_element(By.ID, 'csv-download')
        # download.click()
        time.sleep(5)
        os.system(f'mv ~/Downloads/2008-2022.csv individual-datasets/temp-precip-all/{county}{file_tag}.csv')
        time.sleep(2)
        print(county, 'downloaded')
print('Download complete')

VA-067 downloaded
VA-099 downloaded
VA-159 downloaded
VA-153 downloaded
VA-187 downloaded
VA-015 downloaded
VA-065 downloaded
VA-059 downloaded
VA-003 downloaded
VA-037 downloaded
VA-095 downloaded
VA-089 downloaded
VA-139 downloaded
VA-057 downloaded
VA-179 downloaded
VA-131 downloaded
VA-195 downloaded
VA-053 downloaded
VA-183 downloaded
VA-097 downloaded
VA-163 downloaded
VA-027 downloaded
VA-157 downloaded
VA-107 downloaded
VA-091 downloaded
VA-169 downloaded
VA-167 downloaded
VA-043 downloaded
VA-105 downloaded
VA-127 downloaded
VA-115 downloaded
VA-185 downloaded
VA-121 downloaded
VA-081 downloaded
VA-007 downloaded
VA-175 downloaded
VA-111 downloaded
VA-197 downloaded
VA-155 downloaded
VA-177 downloaded
VA-147 downloaded
VA-137 downloaded
VA-119 downloaded
VA-041 downloaded
VA-045 downloaded
VA-033 downloaded
VA-101 downloaded
VA-143 downloaded
VA-117 downloaded
VA-165 downloaded
VA-085 downloaded
VA-021 downloaded
VA-173 downloaded
VA-113 downloaded
VA-051 downloaded
VA-145 dow

In [ ]:
counties

In [18]:
len(counties)

95

In [21]:
pwd

'/Users/hananur/SchoolWork/Capstone/uvaeicapstone'

In [85]:
temp_dfs = {}
precip_dfs = {}
path = 'individual-datasets/temp-precip-all/'
counties = [str(county) for county in df.County.unique()]
for county in counties:
    temp = pd.read_csv(path+'VA-'+county[2:]+'_t'+'.csv', skiprows=3)
    precip = pd.read_csv(path+'VA-'+county[2:]+'_p'+'.csv', skiprows=3)
    temp_dfs[county] = temp
    precip_dfs[county] = precip

In [86]:
for county in counties:
    for dfs in [temp_dfs, precip_dfs]:
        dfs[county]['Month'] = [int(str(s)[-2:]) for s in dfs[county].Date]
        dfs[county]['Year'] = [str(s)[:4] for s in dfs[county].Date]
        dfs[county] = dfs[county].drop(['Date'], axis=1)
        dfs[county] = dfs[county].pivot_table(index=['Year'], 
                                      columns='Month', 
                                      values='Value')
        dfs[county] = dfs[county].reset_index()
        dfs[county]['County'] = county.title()

In [87]:
temps = pd.concat(temp_dfs).reset_index().drop(['level_0', 'level_1'], axis=1)
temps = temps.rename({i:'T'+str(i) for i in range(1, 13)}, axis=1)
precips = pd.concat(precip_dfs).reset_index().drop(['level_0', 'level_1'], axis=1)
precips = precips.rename({i:'P'+str(i) for i in range(1, 13)}, axis=1)

In [88]:
for tag, df in {'T':temps, 'P':precips}.items():
    df[tag+'Q1'] = df[[tag+str(i) for i in range(1,4)]].aggregate('mean', axis=1)
    df[tag+'Q1'] = df[[tag+str(i) for i in range(1,4)]].aggregate('mean', axis=1)
    df[tag+'Q2'] = df[[tag+str(i) for i in range(4,7)]].aggregate('mean', axis=1)
    df[tag+'Q3'] = df[[tag+str(i) for i in range(7,10)]].aggregate('mean', axis=1)
    df[tag+'Q4'] = df[[tag+str(i) for i in range(10,13)]].aggregate('mean', axis=1)

In [89]:
temps = temps.drop(['T'+str(i) for i in range(1,13)], axis=1)
precips = precips.drop(['P'+str(i) for i in range(1,13)], axis=1)

In [90]:
merged = pd.merge(precips, temps, on=['County', 'Year'])\
            .set_index(['County', 'Year']).reset_index()

In [93]:
merged.to_csv('individual-datasets/temp-precip-all.csv', index=False)

In [95]:
merged.County = merged.County.astype('int64')
merged.Year = merged.Year.astype('int64')
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1425 entries, 0 to 1424
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   County  1425 non-null   int64  
 1   Year    1425 non-null   int64  
 2   PQ1     1425 non-null   float64
 3   PQ2     1425 non-null   float64
 4   PQ3     1425 non-null   float64
 5   PQ4     1425 non-null   float64
 6   TQ1     1425 non-null   float64
 7   TQ2     1425 non-null   float64
 8   TQ3     1425 non-null   float64
 9   TQ4     1425 non-null   float64
dtypes: float64(8), int64(2)
memory usage: 111.5 KB


In [97]:
df = pd.read_csv('individual-datasets/all-county-data-no-tmpprecip.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1425 entries, 0 to 1424
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Year         1425 non-null   int64  
 1   County       1425 non-null   int64  
 2   DSCI         1425 non-null   float64
 3   Income       1080 non-null   float64
 4   Agriculture  1380 non-null   float64
 5   Aquaculture  1380 non-null   float64
 6   Barren       1380 non-null   float64
 7   Developed    1380 non-null   float64
 8   Forest       1380 non-null   float64
 9   Grassland    1380 non-null   float64
 10  Open_Water   1380 non-null   float64
 11  Shrubland    1380 non-null   float64
 12  Wetlands     1380 non-null   float64
 13  VACO2        1330 non-null   float64
dtypes: float64(12), int64(2)
memory usage: 156.0 KB


In [98]:
df = pd.merge(df, merged, on=['Year', 'County'], how='left')
df

,Year,County,DSCI,Income,Agriculture,Aquaculture,Barren,Developed,Forest,Grassland,...,Wetlands,VACO2,PQ1,PQ2,PQ3,PQ4,TQ1,TQ2,TQ3,TQ4
0,2008,51067,181.981132,31454.0,0.120512,0.0,0.000806,0.042941,0.663460,0.145732,...,0.000005,116.763209,1.823333,3.873333,3.786667,2.576667,41.366667,63.966667,71.433333,46.233333
1,2009,51067,27.865385,30916.0,0.103555,0.0,0.001854,0.042546,0.662633,0.161879,...,0.000007,105.907699,2.983333,5.226667,3.793333,6.010000,39.700000,64.066667,70.966667,46.733333
2,2010,51067,27.653846,31328.0,0.087572,0.0,0.001828,0.040608,0.660032,0.181759,...,0.000000,111.886560,4.456667,2.760000,5.240000,2.973333,37.833333,67.266667,74.533333,45.100000
3,2011,51067,50.346154,33190.0,0.088977,0.0,0.002897,0.042279,0.672473,0.166330,...,0.000031,102.519400,2.716667,4.246667,3.740000,4.873333,40.400000,65.700000,73.800000,48.900000
4,2012,51067,62.115385,34500.0,0.086611,0.0,0.003506,0.042493,0.673071,0.166485,...,0.000031,98.562584,2.663333,3.630000,3.636667,2.056667,45.600000,65.000000,73.266667,47.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1420,2018,51093,16.884615,52662.0,0.286654,0.0,0.004196,0.075865,0.294034,0.008047,...,0.206370,110.056786,2.970000,5.110000,5.976667,4.516667,43.300000,68.600000,78.200000,52.533333
1421,2019,51093,11.094340,55316.0,0.288809,0.0,0.000659,0.070008,0.373395,0.004695,...,0.220201,106.794626,4.163333,5.316667,4.070000,3.666667,44.866667,70.300000,77.900000,53.033333
1422,2020,51093,0.826923,57943.0,0.284157,0.0,0.006378,0.070222,0.370450,0.003538,...,0.224667,98.175849,4.426667,4.543333,8.326667,5.273333,49.200000,65.433333,77.300000,53.966667
1423,2021,51093,44.211538,62139.0,0.292600,0.0,0.000701,0.081463,0.354811,0.015733,...,0.218629,97.953403,4.746667,3.590000,6.200000,1.783333,44.333333,66.600000,76.966667,55.466667


In [99]:
df.to_csv('individual-datasets/all-county-data.csv', index=False)